## <b> NB02 - Raininess Index </b>

My Raininiess Index (RI) will be composed of the variable <b> rain sum </b>

This will help me make deductions regarding my two hypotheses:

<b> H0: </b>  London is no different than any other generally-rainy city

<b> H1: </b> London is as rainy as the movies make it out to be

However, I have a third hypothesis:

<b> H2: </b> London is not rainy but cloudy, which gives it a rainy reputation

Of course, H2 cannot be directly proven due to the many, many extraneous factors that go into making a statement like that. This is why I'm leaving "cloud cover" separate from my RI but still using it to make comparisons.

My personal theory is that the gloomy, cloudy weather might be playing a role in the rainy reputation of London, which is why I still chose to use cloud cover as a variable and included it in my hypothesis.

In [2]:
import pandas as pd
import os
import json
import requests
import numpy as np

from functions import *

from lets_plot import *
LetsPlot.setup_html()

In [3]:
daily_rain_df = pd.read_json('../data/daily_rain.json')

daily_rain_df = daily_rain_df.rename(columns={'rain_sum': 'raininess'})

daily_rain_df.to_csv('../data/london_rain.csv', index=False)

In [5]:
all_city_rain_df = pd.read_csv('../data/historical_city_rain_data.csv')

all_city_rain_df = all_city_rain_df.rename(columns={'rain_sum': 'raininess'})

all_city_rain_df.to_csv('../data/historical_city_rain_data.csv')

you can rank it
- every single day, what is the city no 1
- how many times London 1